In [31]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [32]:
mnist=tf.keras.datasets.mnist
(train_images,train_labels),(test_images,test_labels)=mnist.load_data()

train_num=int(len(train_images)*0.8)
train_x=train_images[:train_num]
train_y=train_labels[:train_num]
valid_x=train_images[train_num:]
valid_y=train_labels[train_num:]
test_x=test_images
test_y=test_labels

train_x=train_x.reshape(-1,784)
train_y=tf.one_hot(train_y,depth=10)
valid_x=valid_x.reshape(-1,784)
valid_y=tf.one_hot(valid_y,depth=10)
test_x=test_x.reshape(-1,784)
test_y=tf.one_hot(test_y,depth=10)

train_x=tf.cast(train_x/255.0,tf.float32)
valid_x=tf.cast(valid_x/255.0,tf.float32)
test_x=tf.cast(test_x/255.0,tf.float32)

In [33]:
w1=tf.Variable(tf.random.normal([784,64],mean=0.0,stddev=1.0,dtype=tf.float32))
b1=tf.Variable(tf.zeros([64]),dtype=tf.float32)
w2=tf.Variable(tf.random.normal([64,32],mean=0.0,stddev=1.0,dtype=tf.float32))
b2=tf.Variable(tf.zeros([32]),dtype=tf.float32)
w3=tf.Variable(tf.random.normal([32,10],mean=0.0,stddev=1.0,dtype=tf.float32))
b3=tf.Variable(tf.zeros([10]),dtype=tf.float32)
w=[w1,w2,w3]
b=[b1,b2,b3]

In [34]:
def model(x,w,b):
    x=tf.matmul(x,w[0])+b[0]
    x=tf.nn.relu(x)
    x=tf.matmul(x,w[1])+b[1]
    x=tf.nn.relu(x)
    x=tf.matmul(x,w[2])+b[2]
    pred=tf.nn.softmax(x)
    return pred
def loss(x,y,w,b):
    pred=model(x,w,b)
    loss_=tf.keras.losses.categorical_crossentropy(y_true=y,y_pred=pred)
    return tf.reduce_mean(loss_)
def grad(x,y,w,b):
    with tf.GradientTape() as tape:
        loss_=loss(x,y,w,b)
    return tape.gradient(loss_,w+b)
def accuracy(x,y,w,b):
    pred=model(x,w,b)
    correct_prediction=tf.equal(tf.argmax(pred,1),tf.argmax(y,1))
    return tf.reduce_mean(tf.cast(correct_prediction,tf.float32))  

In [35]:
learning_rate=0.001
optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)

loss_list_train=[]
loss_list_valid=[]
acc_list_train=[]
acc_list_valid=[]

In [36]:
train_epochs=5
batch_size=50
steps=int(train_num/batch_size)
for epoch in range(train_epochs):
    for step in range(steps):
        xs=train_x[step*batch_size:(step+1)*batch_size]
        ys=train_y[step*batch_size:(step+1)*batch_size]
        grads=grad(xs,ys,w,b)
        optimizer.apply_gradients(zip(grads,w+b))

    loss_list_train.append(loss(train_x,train_y,w,b).numpy())
    loss_list_valid.append(loss(valid_x,valid_y,w,b).numpy())
    acc_list_train.append(accuracy(train_x,train_y,w,b).numpy())
    acc_list_valid.append(accuracy(valid_x,valid_y,w,b).numpy())
    print('epoch={:2d}, train loss={:.4f}, train accuracy={:.4f}, value loss={:.4f}, value accuracy=:{:.4f}'
    .format(epoch+1,loss_list_train[-1],acc_list_train[-1],loss_list_valid[-1],acc_list_valid[-1]))

epoch= 1, train loss=7.6598, train accuracy=0.5125, value loss=7.4909, value accuracy=:0.5235
epoch= 2, train loss=4.5918, train accuracy=0.7057, value loss=4.5323, value accuracy=:0.7113
epoch= 3, train loss=3.9995, train accuracy=0.7450, value loss=4.0040, value accuracy=:0.7442
epoch= 4, train loss=3.5897, train accuracy=0.7714, value loss=3.6166, value accuracy=:0.7702
epoch= 5, train loss=3.3548, train accuracy=0.7871, value loss=3.4307, value accuracy=:0.7814
